In [4]:
!pip install neo4j
!pip install bs4

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=5b71d48274144294720a5961761b8e854c733b558e463c7e4d95c92ced84c9da
  Stored in directory: /Users/seol-yoohwan/Library/Caches/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [16]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import re

""" 네이버 랭킹 뉴스 긁어오기 """
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'}
d_list = []
start_data = 20210201
end_data = 20210228
for date_int in range(start_data, end_data):
    date = str(date_int)
    url = "https://news.naver.com/main/ranking/popularDay.nhn?date=" + date
    html = requests.get(url, headers=headers).text
    soup = BS(html, 'html.parser')
    ranking_total = soup.find_all(class_='rankingnews_box')

    for item in ranking_total:
        media = item.a.strong.text
        news = item.find_all(class_="list_content")
        for new in news:
            d = {}
            d['media'] = media
            d['src'] = "https://news.naver.com/" + new.a['href']
            d['title'] = new.a.text
            d['date'] = date
            d_list.append(d)
df = pd.DataFrame(d_list)

In [17]:
df

,media,src,title,date
0,국민일보,https://news.naver.com/https://n.news.naver.co...,진달래 학폭 폭로에 “영혼 털겠다”…소속사 경솔 대응,20210201
1,국민일보,https://news.naver.com/https://n.news.naver.co...,"내일부터 ‘기온 뚝’, 3~4일 수도권 중심으로 눈내려요",20210201
2,국민일보,https://news.naver.com/https://n.news.naver.co...,‘3000’ 붕괴 ‘주린이’들 어쩌나… 고점 기준 7~8% 손실,20210201
3,국민일보,https://news.naver.com/https://n.news.naver.co...,“가장 치욕스런 검사” 中코로나 항문검사에 ‘뒤뚱뒤뚱’,20210201
4,국민일보,https://news.naver.com/https://n.news.naver.co...,메뚜기 상견례… 릴레이 귀향… 그래도 삶은 계속된다,20210201
...,...,...,...,...
8421,중앙SUNDAY,https://news.naver.com/https://n.news.naver.co...,파월의 저금리 약속에도 시장은 “결국은 긴축”에 베팅,20210227
8422,중앙SUNDAY,https://news.naver.com/https://n.news.naver.co...,백신 맞은 뒤 ‘30·30·30 원칙’ 지켜야 안전,20210227
8423,중앙SUNDAY,https://news.naver.com/https://n.news.naver.co...,“남편과 같이 본 하늘”…사진에 담은 송영숙 회장 ‘사부곡’,20210227
8424,레이디경향,https://news.naver.com/https://n.news.naver.co...,울릉도 '울고' 양양군 '웃고'···코로나19가 관광지형도 바꿨다,20210227


In [18]:
df['media'].unique()

array(['국민일보', 'JTBC', '매일경제', '아시아경제', '연합뉴스', '조선일보', '조선비즈', '한겨레',
       '한국경제', '동아일보', 'SBS', '중앙일보', '머니투데이', '미디어오늘', '이데일리', '헤럴드경제',
       '서울경제', '서울신문', 'MBN', '경향신문', 'KBS', '한국일보', '블로터', '파이낸셜뉴스',
       'MBC', '오마이뉴스', '세계일보', '한국경제TV', '부산일보', 'TV조선', 'YTN', '전자신문',
       '뉴스1', '채널A', '뉴시스', '노컷뉴스', 'SBS Biz', 'ZDNet Korea', '데일리안',
       '시사IN', '머니S', '연합뉴스TV', '문화일보', '디지털타임스', '매일신문', '주간조선', '프레시안',
       '더팩트', '강원일보', '조세일보', '헬스조선', '디지털데일리', '아이뉴스24', '코메디닷컴',
       '매경이코노미', '일다', '시사저널', '주간동아', '한경비즈니스', '월간 산', '뉴스타파',
       '코리아중앙데일리', '한겨레21', '여성신문', '동아사이언스', '주간경향', '코리아헤럴드', '기자협회보',
       '신동아', '중앙SUNDAY', '레이디경향'], dtype=object)

In [19]:
df.columns = ['media', 'src', 'title', 'date']

""" 필요 없는 문자 제거 """
def clean_text(row):
    text = row['title']
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("E-mail제거 : " , text , "\n")
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("URL 제거 : ", text , "\n")
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("한글 자음 모음 제거 : ", text , "\n")
    pattern = '<[^>]*>'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("태그 제거 : " , text , "\n")
    pattern = r'\([^)]*\)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("괄호와 괄호안 글자 제거 :  " , text , "\n")
    pattern = '[^\w\s]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("특수기호 제거 : ", text , "\n" )
    pattern = '[^\w\s]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("필요없는 정보 제거 : ", text , "\n" )
    pattern = '["단독"]'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '["속보"]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("단독 속보 제거 : ", text , "\n" )
    text = text.strip()
    # print("양 끝 공백 제거 : ", text , "\n" )
    text = " ".join(text.split())
    # print("중간에 공백은 1개만 : ", text )
    return text

df['title'] = df.apply(clean_text, axis=1)

In [28]:
# macy
!pip install konlpy

In [29]:
""" 키워드 추출 from title """
from konlpy.tag import Kkma
from konlpy.tag import Komoran

kkma = Kkma()
komoran = Komoran()

df['keyword'] = ''

df = df.reset_index(drop=True)

for idx_line in range(len(df)):
    try:
        nouns_list = komoran.nouns(df['title'].loc[idx_line])
    except:
        continue
     # 한글자는 이상한게 많아서 2글자 이상
    nouns_list_c = [nouns for nouns in nouns_list if len(nouns) > 1]   
    df.loc[idx_line]['keyword'] = set(nouns_list_c)

 # 코리아헤럴드는 영어 제목임
df = df[df['media'] != '코리아헤럴드'] 
# 주간경향은 같은 title이 많음  
df = df[df['media'] != '주간경향']    

JVMNotFoundException: No JVM shared library file (libjli.dylib) found. Try setting up the JAVA_HOME environment variable properly.

In [ ]:
from neo4j import GraphDatabase


""" make node & relationship"""

def add_article(tx, title, date, media, keyword):
    tx.run("MERGE (a:Article {title: $title , date: $date, media: $media, keyword: $keyword})",
           title=title, date=date, media=media, keyword=keyword)


def add_media(tx):
    tx.run("MATCH (a:Article) "
           "MERGE (b:Media {name:a.media}) "
           "MERGE (a)<-[r:Publish]-(b)")


def add_keyword(tx):
    tx.run("MATCH (a:Article) "
           "UNWIND a.keyword as k "
           "MERGE (b:Keyword {name:k}) "
           "MERGE (a)-[r:Include]->(b)")



""" 한자와 공백 제거 """
# Neo4j -> Gephi 에서 parsing error의 원인이 될 수 있음
def clean_text_for_neo4j(row):
    text = row['title_c']
    text = re.sub(pattern='[^a-zA-Z0-9ㄱ-ㅣ가-힣]', repl='', string=text)
    # print("영어, 숫자, 한글만 포함 : ", text )
    return text

df['title_c_neo4j'] = df.apply(clean_text_for_neo4j, axis=1)

""" 연결 """
# Neo4j 브라우저에서 설정한 계정의 ID, PASSWORD를 통해 접속
greeter = GraphDatabase.driver("bolt://localhost:7687", auth=("test", "qwer9514@@")) 

In [ ]:
""" 입력 """
# Cyper code를 이용,  크롤링한 Data를 DB에 입력
with greeter.session() as session:
    """ make node """
    for idx in range(len(df)):
        session.write_transaction(add_article, title=df.iloc[idx]['title_c_neo4j'], date=df.iloc[idx]['date'],
                                  media=df.iloc[idx]['media'], keyword=list(df.iloc[idx]['keyword']))
    session.write_transaction(add_media)
    session.write_transaction(add_keyword)